# KDD Stage 5 — Deployment / Knowledge Extraction

Save the final pipeline, show serving snippet, and example requests.

In [ ]:
import joblib, json
# Assume 'best_pipe' trained on TRAIN+VAL; bundle and save:
bundle = {"pipeline": None, "meta": {"version": "1.0.0"}}
print("In your environment, replace None with the trained pipeline and save as models/best_model.pkl")


**FastAPI serving snippet:**

```python
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field
import joblib, pandas as pd

FEATURES = ["SepalLengthCm","SepalWidthCm","PetalLengthCm","PetalWidthCm"]
bundle = joblib.load("models/best_model.pkl")
pipe = bundle["pipeline"]
CLASSES = list(pipe.named_steps["clf"].classes_)

class IrisRequest(BaseModel):
    SepalLengthCm: float = Field(..., ge=0)
    SepalWidthCm:  float = Field(..., ge=0)
    PetalLengthCm: float = Field(..., ge=0)
    PetalWidthCm:  float = Field(..., ge=0)

app = FastAPI(title="Iris Classifier")
@app.post("/predict")
def predict(item: IrisRequest):
    df = pd.DataFrame([item.dict()])
    proba = pipe.predict_proba(df)[0].tolist()
    pred = CLASSES[int(np.argmax(proba))]
    return {"prediction": pred, "classes": CLASSES, "probabilities": proba}
```

**Example request (curl):**

```bash
curl -H 'Content-Type: application/json' \
  -d '{"SepalLengthCm":5.1,"SepalWidthCm":3.5,"PetalLengthCm":1.4,"PetalWidthCm":0.2}' \
  http://localhost:8080/predict
```